In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
from matplotlib import pyplot as plt

class Option:
    '''
    S: asset_price
    sig: asset_volatility
    K: strike_price
    T: time_to_maturity (year)
    r: risk_free_rate
    types: Call/Put
    '''
    def __init__(self, S, sig, K, T, r, types):
        self._S = S
        self._sig = sig
        self._K = K
        self._T = T
        self._r = r
        self._types = types
        
    def info(self):
        info = {
            "Asset price": self._S,
            "Asset volatility": self._sig,
            "Strike price" : self._K,
            "Time to maturity (year)": self._T,
            "Risk free rate": self._r,
            "Types": self._types
        }
        return info
    
    def _d1(self):
        numerator = (np.log(self._S/self._K)+(self._r+self._sig**2/2.0)*self._T)
        denumerator = (self._sig*np.sqrt(self._T))
        return np.divide(numerator, denumerator, out=np.asarray(numerator*np.Inf), where=denumerator!=0.0)

    def _d2(self):
        return self._d1()-self._sig*np.sqrt(self._T)
        
    def _price(self):
        if self._types == "Call":
            return np.round(self._S*norm.cdf(self._d1()) - self._K*np.exp(-self._r*self._T)*norm.cdf(self._d2()), 2)
        else:
            return np.round(self._K*np.exp(-self._r*self._T)*norm.cdf(-self._d2()) - self._S*norm.cdf(-self._d1()), 2)
    
    def _delta(self):
        if self._types == "Call":
            return np.round(norm.cdf(self._d1()), 3)
        else:
            return np.round(norm.cdf(self._d1()) - 1, 3)
        
    def _gamma(self):
        if self._types == "Call":
            return np.round(norm.cdf(self._d1())/(self._S*self._sig*np.sqrt(self._T)), 3)
        else:
            return np.round(-self._S*norm.cdf(self._d1())*self._sig)/(2*np.sqrt(self._T)) \
                            + self._r*self._K*np.exp(-self._r*self._T)*norm.cdf(-self._d2(), 3)              
    
    def _theta(self):
            return np.round(norm.cdf(self._d1())/(self._S*self._sig*np.sqrt(self._T)), 3)
                            
    def _vega(self):
            return np.round(self._S*np.sqrt(self._T)*norm.cdf(self._d1()), 3)
    
    def _rho(self):
        if self._types == "Call":
            return np.round(self._K*self._T*np.exp(-self._r*self._T)*norm.cdf(self._d2()), 3)
        else:
            return np.round(-self._K*self._T*np.exp(-self._r*self._T)*norm.cdf(-self._d2()), 3)

In [2]:
#Example:

S0 = 49
sig = 0.2
K = 50
T = 140/360 #20 weeks
r = 0.05 
types = "Call"

Option1 = Option(S0, sig, K, T, r, types)

print(Option1._delta())

print(Option1._rho())

0.522
9.015


__Geometric Brownian Motion__
$$
dS = r S dt + \sigma S dw^Q\\
S_t = S_0 exp\Big[\Big(r - \frac{\sigma^2}{2}\Big) t + \sigma Z_t\Big] \ \text{ where } Z_t \sim Niid(0,t)
$$

In [3]:
#Dynamic Delta Hedging
Number_of_options = 100_000
N = 10
dt = np.linspace(0,T,N)
np.random.seed(42)

St = np.round(S0*np.exp((r - sig**2/2)*dt + sig*np.random.normal(0, dt)), 2)

Option_series = Option(St, sig, K, dt[::-1], r, types)

df = pd.DataFrame(np.transpose([St,Option_series._delta()]), columns=["Price", "Delta"])
df["Shares purchased"] = (df["Delta"] - df["Delta"].shift(1))*Number_of_options
df["Shares purchased"].iloc[0] = df["Delta"].iloc[0]*Number_of_options
df["Total of shares"] = np.round(df["Shares purchased"].cumsum())
df["Total of shares"].iloc[-1] = 0
df["Shares purchased"].iloc[-1] = -df["Total of shares"].iloc[-2]
df["Cost of purchase"] = df["Price"]*df["Shares purchased"]
if df["Price"].iloc[-1] > K:
    df["Cost of purchase"].iloc[-1] = K*df["Shares purchased"].iloc[-1]
df["Interest"] = np.round(df["Cost of purchase"].cumsum()*r*T/(20), 2)
df["Interest"] = df["Interest"].shift(1)
df["Interest"].iloc[0] = 0
df["Cummulative cost including interest"] = (df["Cost of purchase"] + df["Interest"]).cumsum()
df

,Price,Delta,Shares purchased,Total of shares,Cost of purchase,Interest,Cummulative cost including interest
0,49.00,0.522,52200.0,52200.0,2557800.0,0.00,2557800.00
1,49.00,0.514,-800.0,51400.0,-39200.0,2486.75,2521086.75
2,49.68,0.553,3900.0,55300.0,193752.0,2448.64,2717287.39
3,51.17,0.657,10400.0,65700.0,532168.0,2637.01,3252092.40
4,48.86,0.466,-19100.0,46600.0,-933226.0,3154.39,2322020.79
5,48.82,0.444,-2200.0,44400.0,-107404.0,2247.09,2216863.88
6,53.60,0.862,41800.0,86200.0,2240480.0,2142.67,4459486.55
7,51.80,0.759,-10300.0,75900.0,-533540.0,4320.92,3930267.47
8,47.93,0.173,-58600.0,17300.0,-2808698.0,3802.20,1125371.67
9,51.71,1.000,-17300.0,0.0,-865000.0,1071.52,261443.19


In [4]:
#Check
Number_of_options*Option_series._price()[0] 

242000.0

In [5]:
(df["Cummulative cost including interest"].iloc[-1])*math.exp(-T*r)

256408.6776865531